<a href="https://colab.research.google.com/github/mhpbreugem/WP1/blob/main/bbp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
using Distributions, Random, NLsolve, LinearAlgebra, Printf, Optim, BenchmarkTools

In [2]:
# Chapter 0: Parameters

# Economic parameters
const β = 0.95
const γ = 4.0
const e0 = 3.0
const EΠ = 1.0
const Π0 = 1.0
const θXbar = 1.0
const θYbar = 1.0
const μθX = 0.0
const σθX = 0.2
const μθY = 0.0
const σθY = 0.1
const Nσπ = 2
const σΠ = 0.4
const ρ = 0
const W0 = e0

const ΞЮ = [repeat([x], 1) for x in range(0.1, stop=1.0, length=10)]

# Grid parameters
const Nσπ = 17
const NNσπ = 1
const Nπ = 2 * Nσπ * NNσπ + 1  # number of grid points for payoff

const NσΠ = 2
const NNσΠ = 1
const NΠ = 2 * NσΠ * NNσΠ + 1  # number of grid points for payoff

const NθX = 5
const NθY = 5
const NσθX = 2
const NσθY = 2

# Derived midpoints
const μπi = (Nπ + 1) ÷ 2
const μΠi = (NΠ + 1) ÷ 2
const μθXi = (NθX + 1) ÷ 2

# Total number of states
const NS = Nπ
const NN = NθX * NθY * NΠ

125

In [3]:
# Chapter 1: Grid Construction

# Conjectured Payoffs
const ππ = collect(range(EΠ - Nσπ*σΠ, EΠ + Nσπ*σΠ, length=Nπ))
const πω = pdf.(Normal(EΠ, σΠ), ππ)
const πΩ = πω ./ sum(πω)

# Payoff Parameters
const Δ1 = Π0
const Δ2 = ππ

# True Payoffs
const ΞΠ = EΠ .+ σΠ .* range(-NσΠ, NσΠ, length=NΠ)
const ππ_to_index = Dict(round(ππ[πi], digits=8) => πi for πi in 1:Nπ)
const ΞΠi = [ππ_to_index[round(ΞΠ[Πi], digits=8)] for Πi in 1:NΠ]

# Signals
const S = ππ

@inline function ψϵω(Ю, ϵ)
    σϵ = 1 / sqrt(Ю)
    pdf(Normal(0.0, σϵ), ϵ)
end

function φSΩπ(Ю)
    σϵ = 1 / sqrt(Ю[1])
    SΩ = [pdf(Normal(π, σϵ), s) for π in ππ, s in ππ]
    SΩnorm = SΩ ./ sum(SΩ, dims=2)
    return SΩnorm
end

# Noise Trader Demand
const ΞθX = μθX .+ σθX .* range(-NσθX, NσθX, length=NθX)
const ΞθY = μθY .+ σθY .* range(-NσθY, NσθY, length=NθY)

const Σθ = [σθX^2 σθX*σθY*ρ; σθX*σθY*ρ σθY^2]
const dist = MvNormal([μθX, μθY], Σθ)
θXYωω(x, y) = pdf(dist, [x, y])

θXYωω (generic function with 1 method)

In [4]:
# Chapter 2: Learning

# Full learning (private + public information)
function φΞΩI(Ю, XΣ, YΣ)
    ϵ = S .- ππ'
    ψϵ_values = ψϵω.(Ю, ϵ)
    θXY_values = θXYωω.(θXbar .- XΣ, θYbar .- YΣ)

    ΞΩI = πΩ .* ψϵ_values .* θXY_values
    sum_ΞΩI = sum(ΞΩI, dims=1)
    ΞΩI .= ΞΩI ./ sum_ΞΩI

    return ΞΩI
end

φΞΩI (generic function with 1 method)

In [5]:
# Chapter 3: System of Equations
const ΣW0 = W0

function φSYS(Ж, Ю, Б)
    X = Ж[1:NS]
    C1 = Ж[NS+1:2*NS]
    PX, PY = Ж[2*NS+1:2*NS+2]

    Πi = Б[1]
    θX = ΞθX[Б[2]]
    θY = ΞθY[Б[3]]

    # Conditional Expected Quantities
    SΩπ = φSΩπ(Ю)
    SΩ_X_sum = sum(SΩπ .* X', dims=2)
    SΩ_C1_sum = sum(SΩπ .* C1', dims=2)

    # Aggregate Demand
    XΣ = SΩ_X_sum
    YΣ = (ΣW0 .- PX .* XΣ .- SΩ_C1_sum) ./ PY

    # Learning
    Ω = φΞΩI(Ю, XΣ, YΣ)

    # FOC's
    C2_values = (W0 .- PX .* X' .- C1') ./ PY .+ X' .* Δ2
    FOCX_values = β .* sum(Ω .* exp.(-γ .* C2_values) .* (-PX ./ PY .+ Δ2), dims=1)
    FOCC1_values = exp.(-γ .* C1') + β .* sum(Ω .* exp.(-γ .* C2_values) .* (-1.0 ./ PY), dims=1)

    # Market Clearing
    MCX = XΣ[ΞΠi[Πi]] - (θXbar - θX)
    MCY = YΣ[ΞΠi[Πi]] - (θYbar - θY)

    return vcat(FOCX_values', FOCC1_values', MCX, MCY)
end

φSYS (generic function with 1 method)

In [6]:
# Chapter 4: Zero-Info Starting Point (ξ)
const W0ξ = W0

# Fixed Point Zero info
function φSYSξ(Жξ,Б)
    PXξ, PYξ = Жξ[1:2]
    θX = ΞθX[Б[2]]
    θY = ΞθY[Б[3]]

    # Direct solution of MC's
    Xξ = (θXbar - θX)
    C1ξ = W0ξ - PXξ * (θXbar - θX) - PYξ *(θYbar - θY)

    # FOC's
    C2ξ_values = (W0ξ - PXξ * Xξ - C1ξ) / PYξ  .+ Xξ .* Δ2
    FOCXξ_values = β .* sum(πΩ .* exp.(-γ .* C2ξ_values) .* (-PXξ ./ PYξ .+ Δ2), dims=1)
    FOCC1ξ_values = exp.(-γ .* C1ξ) .+ β .* sum(πΩ .* exp.(-γ .* C2ξ_values) .* (-1.0 ./ PYξ), dims=1)

    return vcat(FOCXξ_values, FOCC1ξ_values)
end

# Starting Point of Full Code
function φЖ0(Б)
    θXi, θYi = Б[2], Б[3]
    θX = ΞθX[θXi]
    θY = ΞθY[θYi]
    solξ = nlsolve(Жξ -> φSYSξ(Жξ, [0, θXi, θYi]), [0.6, 0.7])
    PXξ, PYξ = solξ.zero
    Ж = zeros(2 * NS + 2)
    Ж[1:NS] .= θXbar - θX
    Ж[NS+1:2*NS] .= W0ξ .- PXξ * (θXbar .- θX) .- PYξ * (θYbar .- θY)
    Ж[2*NS+1:2*NS+2] .= [PXξ, PYξ]
    return Ж
end


φЖ0 (generic function with 1 method)

In [7]:
# Chapter 5: Solution over grid

sol_zeros = Array{Array{Float64}, 3}(undef, NΠ, NθX, NθY)

for Πi in 1:NΠ
    for θXi in 1:NθX
        for θYi in 1:NθY
            Ж_ini = φЖ0([Πi, θXi, θYi])
            for Ю_value in range(0.1, stop=1, length=7)
                print("\rProcessing: Πi = $Πi, θXi = $θXi, θYi = $θYi, Ю_value = $Ю_value")
                flush(stdout)
                sol = nlsolve(Ж -> φSYS(Ж, [Ю_value], [Πi, θXi, θYi]), Ж_ini)
                Ж_ini = sol.zero
            end
        end
    end
end


Processing: Πi = 5, θXi = 5, θYi = 5, Ю_value = 1.0



---

